In [1]:
pip install psycopg2-binary spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3

In [2]:
import psycopg2

# Connect to PostgreSQL
def connect_to_db():
    conn = psycopg2.connect(
        host="localhost",
        database="mydb",
        user="surajnag",
        password="suraj"
    )
    return conn

# Querying the database
def query_database(query, params=None):
    conn = connect_to_db()
    cur = conn.cursor()
    cur.execute(query, params)
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results


In [4]:
import spacy

import spacy.cli
spacy.cli.download("en_core_web_sm")

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Sample query
def extract_entities(user_query):
    doc = nlp(user_query)
    for ent in doc.ents:
        print(f'{ent.text}: {ent.label_}')
    
extract_entities("What were the sales for Smartphone last month?")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.6 MB/s eta 0:00:00a 0:00:010m
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for spacy: filename=spacy-3.7.6-cp311-cp311-macosx_14_0_arm64.whl size=6209951 sha256=51c993f72ef7cca336be795ba954f6dfc76f0e3631435be561656a14fb5af1d7
  Stored in directory: /Users/surajnag/Library/Caches/pip/wheels/d4/20/60/0719a53a3a30f17c67a2beb43d9b4a4cc30751716d1f4a2416
Successfully built spacy
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.0
    Uninstalling spacy-3.8.0:
      Successfully uninstalled spacy-3.8.0



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/opt/homebrew/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


last month: DATE


In [7]:
def generate_sql(user_query):
    # Basic pattern matching
    if "sales" in user_query.lower():
        if "product" in user_query.lower():
            product_name = "Smartphone"  # Extracted from NLP step
            query = """
                SELECT SUM(total_amount) FROM sales 
                WHERE product_name = 'Smartphone'
                AND sale_date > CURRENT_DATE - INTERVAL '1 month'
            """
            return query_database(query, (product_name,))
    
    return "Sorry, I don't understand that query."

# Sample question
result = generate_sql("What were the sales for Product A last month?")
print(result)


[(Decimal('3000.00'),)]
